# Desde Selenium entro la web de IMDB. Más adelante este proceso se hará directamente desde IMDB, pero para el MPV podemos hacerlo desde cero: entrando en la URL de la película de IMDB que quiero explorar.

# Busco y almaceno los 10 primeros actores de esta pelicula y consigo sus enlaces de IMDB.

# Desde Beautiful Soup, recorro esas páginas y extraigo los datos:

# Nombre. Año de nacimiento. ¿Por qué es conocido? ¿Qué hace? Premios.

# Los almaceno



In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Importo librería de Carla

from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.


In [12]:
# Creo una función que recibe la URL de una película en IMDB y devuelve un diccionario con los nombres 
# de los 10 intérpretes principales y sus URL en IMDB.

def conseguir_url_actores (url_pelicula):
    # Activo Selenium y abro un navegador para explorar la web de la película.
    driver = webdriver.Chrome()
    driver.get(url_pelicula)
    print("Entrando en la url de la película.")
    sleep(2)
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
    print("Aceptando cookies.")
    sleep(2)
    print("Maximizo ventana.")
    sleep(2)
    # Quiero encontrar el selector del casting. Y después iterarlo para recopilar el nombre y la url de imdb de cada intérprete.

    elemento_casting = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid")
    print("Extraigo el elemento que tiene a todo el casting")
    sleep(2)

    actores_url = {} # Inicializo el diccionario que almacenará las URL de cada intérprete.

    for i in range (1, 11):
        actor = elemento_casting.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-bfec09a1-7.kSFMrr > a")
        # print(f"La url del actor {actor.text} es {actor.get_attribute("href")}")
        actores_url[actor.text] = actor.get_attribute("href")

    sleep(2)
    driver.quit()
    print("Cerrando navegador.  Exploración ejecutada correctamente. Disfrute de sus URL.")

    # Mediante los selectores css he conseguido las direcciones de cada actor, que usaré para extraer el resto de datos con Beautiful Soup.

    # "NOTA PARA MÍ: Me gustaría aprender a hacer lo mismo entrando por elementos y atributos, no solo por css")

    """¿Cómo he conseguido el selector para cada intérprete? Sacando el inspector las líneas con la información de cada actor y luego haciendo esto dinámico.
    1: #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.sc-bfec09a1-7.kSFMrr > a
    2: #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(2) > div.sc-bfec09a1-7.kSFMrr > a
    3: #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(3) > div.sc-bfec09a1-7.kSFMrr > a
    """ 

    return actores_url.items()
    

# Hacemos una prueba de que funciona. Llamamos a la funcion

url_jurassic_park = "https://www.imdb.com/title/tt0107290/?ref_=nv_sr_srsg_0_tt_8_nm_0_in_0_q_jurassic%2520park"
diccionario_url =  conseguir_url_actores (url_jurassic_park)
print(f"\n Tras llamar a la funcion conseguir_url_actores hemos extraído los siguientes datos:")
data = pd.DataFrame(diccionario_url)
print(data)




Entrando en la url de la película.
Aceptando cookies.
Maximizo ventana.
Extraigo el elemento que tiene a todo el casting
Cerrando navegador.  Exploración ejecutada correctamente. Disfrute de sus URL.

 Tras llamar a la funcion conseguir_url_actores hemos extraído los siguientes datos:
                      0                                                  1
0             Sam Neill  https://www.imdb.com/name/nm0000554/?ref_=tt_c...
1            Laura Dern  https://www.imdb.com/name/nm0000368/?ref_=tt_c...
2         Jeff Goldblum  https://www.imdb.com/name/nm0000156/?ref_=tt_c...
3  Richard Attenborough  https://www.imdb.com/name/nm0000277/?ref_=tt_c...
4              Bob Peck  https://www.imdb.com/name/nm0669629/?ref_=tt_c...
5        Martin Ferrero  https://www.imdb.com/name/nm0274684/?ref_=tt_c...
6               BD Wong  https://www.imdb.com/name/nm0000703/?ref_=tt_c...
7       Joseph Mazzello  https://www.imdb.com/name/nm0001515/?ref_=tt_c...
8       Ariana Richards  https://www.im